# **Importation d'instances**

In [103]:
%%capture
import random
import math
import numpy as np
import pandas as pd
!pip install smac
items = [4,7,8,5,6,5,2,3]
n=len(items)
c = 10

# **First-Fit-Decreasing**

In [104]:
def heuristic_FFD(c,w):
  n = len(w)
  order = sorted([i for i in range(n)], key = lambda i:w[i],reverse=True)
  bin_for_item = [-1 for i in range(n)]
  bin_space = []
  for i in order:
    for j in range(len(bin_space)):
      if w[i]<bin_space[j]:
        bin_for_item[i]=j
        bin_space[j]-=w[i]
        break
    if bin_for_item[i] < 0:
      j = len(bin_space)
      bin_for_item[i] = j
      bin_space.append(c-w[i])
  n_bin = len(bin_space)
  return n_bin, bin_for_item

print(heuristic_FFD(c,items))  

(5, [3, 1, 0, 3, 2, 4, 1, 2])


# **First-Fit-Increasing**

In [105]:
def heuristic_FFI(c,w):
  n = len(w)
  order = sorted([i for i in range(n)],key = lambda i:w[i])
  bin_for_item = [-1 for i in range(n)]
  bin_space = []
  for i in order:
    for j in range(len(bin_space)):
      if w[i]<bin_space[j]:
        bin_for_item[i]=j
        bin_space[j]-=w[i]
        break
    if bin_for_item[i] < 0:
      j = len(bin_space)
      bin_for_item[i] = j
      bin_space.append(c-w[i])
  n_bin = len(bin_space)
  return n_bin, bin_for_item

print(heuristic_FFI(c,items))

(6, [0, 4, 5, 1, 3, 2, 0, 0])


# **Best-Fit**

In [106]:
def heuristic_BF(c,w):
  n = len(w)
  bin_for_item = [-1 for i in range(n)]
  bin_space = []
  for i,wi in enumerate(w):
    tmp = sorted(bin_space)
    k=0
    while len(tmp)!=0:
      #k = bin_space.index(min(bin_space))
      if wi < tmp[k]:
        j = bin_space.index(tmp[k])
        bin_for_item[i]=j
        bin_space[j]-=wi
        break
      else:
        k+=1
        if (k == len(bin_space)): break
      
    if bin_for_item[i] < 0:
      j = len(bin_space)
      bin_for_item[i] = j
      bin_space.append(c-wi)
  n_bin = len(bin_space)
  return n_bin, bin_for_item

print(heuristic_BF(c,items))

(5, [0, 1, 2, 0, 3, 4, 1, 3])


# **Worst-Fit**

In [107]:
def heuristic_WF(c,w):
  n = len(w)
  bin_for_item = [-1 for i in range(n)]
  bin_space = []
  for i,wi in enumerate(w):
    if (bin_space != []):
      k = bin_space.index(max(bin_space))
      if wi < bin_space[k]:
        bin_for_item[i]=k
        bin_space[k]-=wi
      
    if bin_for_item[i] < 0:
      j = len(bin_space)
      bin_for_item[i] = j
      bin_space.append(c-wi)
  n_bin = len(bin_space)
  return n_bin, bin_for_item

print(heuristic_WF(c,items))   

(5, [0, 1, 2, 0, 3, 4, 4, 3])


# **Almost-Worst-Fit**

In [108]:
def heuristic_AWF(c,w):
  n = len(w)
  bin_for_item = [-1 for i in range(n)]
  bin_space = []
  for i,wi in enumerate(w):
    tmp = bin_space
    if (len(tmp)!=0):
      k = tmp.index(max(tmp)); tmp[k] = 0; k = tmp.index(max(tmp))
      if w[i]<bin_space[k]:
        bin_for_item[i]=k
        bin_space[k]-=wi
      
    if bin_for_item[i] < 0:
      j = len(bin_space)
      bin_for_item[i] = j
      bin_space.append(c-wi)
  n_bin = len(bin_space)
  return n_bin, bin_for_item

print(heuristic_AWF(c,items)) 

(8, [0, 1, 2, 3, 4, 5, 6, 7])


# **Next-Fit**

In [109]:
def heuristic_NF(c,w):
  n = len(w)
  bin_for_item = [-1 for i in range(n)]
  bin_space = []
  for i,wi in enumerate(w):
    for j in range(len(bin_space),0,-1):
      if wi<bin_space[j-1]:
        bin_for_item[i]=j-1
        bin_space[j-1]-=wi
        break
    if bin_for_item[i] < 0:
      j = len(bin_space)
      bin_for_item[i] = j
      bin_space.append(c-wi)
  n_bin = len(bin_space)
  return n_bin, bin_for_item

print(heuristic_NF(c,items))  

(5, [0, 1, 2, 0, 3, 4, 4, 3])


# **BRANCH & BOUND**

**Representation de l'espace des solutions :**
on prend une liste qui designe pour chaque item le bin a qui il est affecté, on aura comme noeud racine [ -1 ... ].
( le num du bin au maximum peu atteindre n dans le pire cas) 


*stratégie en profondeur*:  les premiere solution sont les meilleur pas la peine de tout évaluer

**Fonction d'evaluation**

In [110]:
def eval(conf):
  return max(conf)+1
  #if (nbin == -1): config impossible

In [111]:
def verif(conf):
  # if -1 in conf : return 0
  # else: 
  en = enumerate(conf)
  bin = set(conf)
  for b in bin:
    indices = [index for index, element in enumerate(conf) if element == b]
    s = sum([items[i] for i in indices])
    if s > c: return 0

  #verifier que les numero des bins sont bien séquentiel partant de 0
  mx = max(conf)
  lis = [i for i in range(mx)] 
  for i in lis:
    if (i not in list(set(conf))): 
      return 0
  return 1 

In [112]:
def Branch_Bound(items):
  n=len(items)
  Confs=[]
  Confs.append([-1]*n)
  m, soluce = heuristic_FFD(c,items)
  opt= soluce
  print(m, soluce)
  j=0
  while len(Confs)>0 :
    #print(Confs)
    v = Confs.pop(0)
    ev = eval(v)
    if (ev < m):
      j = v.index(-1)
      for i in range(n,-1,-1):
        v = v.copy()
        if (j<n): v[j]=i 
        if -1 in v:
            Confs.insert(0,v)
        else:
          if (verif(v)==1):
            if eval(v) <= m:
              m = eval(v)
              opt = v
              print('Solution trouvé',m,v)
      j+=1  
  return m,opt

#print('Solution optimal: ',m,opt)
 

# **Recuit Simulé**

**Voisinage:** pour faire des mouvement on doit définir la notion de voisinage pour notre probleme, dans se cas un voisin et une affectation differente d'un seul item.

In [113]:
def voisins(conf,ind,max_bin):
  V = []
  for i in range(max_bin):
    if i!=conf[ind]:
      v = conf.copy()
      v[ind]=i
      if verif(v):
        V.append(v)
  return V    
    

**Choix du meilleur voisin:** c'est celui qui contient un bin presque vide pour que le prochain voisin essaye de le vider 

In [114]:
def eval_choix(conf):
  bin = set(conf)
  bin_space = []
  for b in bin:
    indices = [index for index, element in enumerate(conf) if element == b]
    s = sum([items[i] for i in indices])
    bin_space.append(c-s)
  return max(bin_space) 


In [115]:
def choix(voisins):
  if len(voisins)>0:
    evals = [eval_choix(v) for v in voisins]
    choix = evals.index(max(evals))
    return voisins[choix],voisins
  return -1    

In [116]:
def RS(soluce,m,alpha,t,it_palier,k_arret):
  x = soluce; fx = eval_choix(x)
  optimum = soluce
  o = m
  k=0
  while True:
    for i in range(it_palier):
      j = random.randint(0,n-1)
      bv = choix(voisins(x,j,m))
      if bv == -1 : continue
      best , vois = bv
      #v= best;fv = eval_choix(v)
      v = random.choice(vois); fv = eval_choix(v)
      if eval(v) < eval(x) : 
        optimum = v
        o = eval(v)
        fv *= 10
        #k=k_arret-1
        #break
      delta = fv - fx
      if delta > 0:
        x = v
        if eval(x) < o:
          optimum = x
          o = eval(x)
      else:
        u=random.random()  
        if u < math.exp(delta / t):
          x = v
      t = alpha * t 
      if t == 0: 
        t=100
        break    
    k+=1   
    if k == k_arret :break
  return o,optimum

m,soluce = heuristic_FFD(c,items)
alpha = 0.94
t = 100
it_palier = 100
k_arret = 100

RS(soluce,m,alpha,t,it_palier,k_arret)

(4, [3, 1, 0, 2, 3, 2, 0, 1])

In [117]:
def test_RS(x):
  m,soluce = heuristic_FFD(c,items)
  alpha = x['x0']
  t = x['x1']
  it_palier = x['x2']
  k_arret = x['x3']

  o,opt = RS(soluce,m,alpha,t,it_palier,k_arret)
  return o

# **Recherche Taboue**

In [118]:
def bon_voisins(conf,ind,max_bin,taboue,L):
  V = []
  for i in range(max_bin):
    if i!=conf[ind]:
      v = conf.copy()
      v[ind]=i
      if verif(v) and ((v not in taboue) or len(taboue)== L or eval(v) < max_bin):
        if len(taboue)== 10: taboue.pop()
        V.append(v)
  return V        
#bon_voisins(soluce,2,5,[])

In [119]:
def RT(soluce,m,k_arret,L):
  x = soluce; fx = eval_choix(x)
  optimum = soluce
  o = m
  k=0
  Tabou = []
  while True:
    j = random.randint(0,n-1)
    bv = choix(bon_voisins(x,j,m,Tabou,L))
    if bv == -1 : continue
    best, vois = bv
    v= best;fv = eval_choix(v)
    x = v
    Tabou.append(x)
    if eval(x) < o:
      optimum = x
      o = eval(x)
    k+=1   
    if k == k_arret :break
  return o,optimum

m,soluce = heuristic_FFD(c,items)
k_arret = 100
L=10
RT(soluce,m,k_arret,L)

(4, [2, 1, 0, 3, 2, 3, 0, 1])

In [120]:
def test_RT(x):
  m,soluce = heuristic_FFD(c,items)
  k_arret = x['x0']
  L = x['x1']
  o,opt = RT(soluce,m,k_arret,L)
  return o

# **Algorithme genetique**

# **Tuning Parameters**

In [121]:
import logging

import numpy as np
from ConfigSpace.hyperparameters import UniformFloatHyperparameter
from ConfigSpace.hyperparameters import UniformIntegerHyperparameter

# Import ConfigSpace and different types of parameters
from smac.configspace import ConfigurationSpace
from smac.facade.smac_hpo_facade import SMAC4HPO
# Import SMAC-utilities
from smac.scenario.scenario import Scenario



from smac.facade.func_facade import fmin_smac

Tuning hyperparams for RT

In [122]:
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output

# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()
x0 = UniformIntegerHyperparameter("x0", 1, 30, default_value=20)
x1 = UniformIntegerHyperparameter("x1", 1, 30, default_value=10)
#x1 = UniformFloatHyperparameter("x1", -5, 10, default_value=-4)
cs.add_hyperparameters([x0,x1])

# Scenario object
scenario = Scenario({"run_obj": "quality",  # we optimize quality (alternatively runtime)
                     "runcount-limit": 5,  # max. number of function evaluations; for this example set to a low number
                     "cs": cs,  # configuration space
                     "deterministic": "true"
                     })

# Example call of the function
# It returns: Status, Cost, Runtime, Additional Infos
def_value = test_RT(cs.get_default_configuration())
print("Default Value: %.2f" % def_value)

# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC4HPO(scenario=scenario,
                rng=np.random.RandomState(42),
                tae_runner=test_RT)

smac.optimize()

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-04-19_02:38:35_702668
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 1 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent


Default Value: 5.00
Optimizing! Depending on your machine, this might take a few minutes.


INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 5.0000
INFO:smac.intensification.intensification.Intensifier:Challenger (4.0000) is better than incumbent (5.0000) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:
INFO:smac.intensification.intensification.Intensifier:  x0 : 16 -> 26
INFO:smac.intensification.intensification.Intensifier:  x1 : 16 -> 30
INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 0.056916 sec, available: 0.000010 sec)
INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 0.024711 sec, available: 0.000010 sec)
INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 0.025349 sec, avail

Configuration:
  x0, Value: 26
  x1, Value: 30

Tuning hyperparams for RS

In [123]:
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output

# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()
x0 = UniformFloatHyperparameter("x0", 0.8, 0.99, default_value=0.9)
x1 = UniformIntegerHyperparameter("x1", 1, 60, default_value=40)
x2 = UniformIntegerHyperparameter("x2", 1, 5, default_value=2)
x3 = UniformIntegerHyperparameter("x3", 1, 30, default_value=20)
cs.add_hyperparameters([x0,x1,x2,x3])

# Scenario object
scenario = Scenario({"run_obj": "quality",  # we optimize quality (alternatively runtime)
                     "runcount-limit": 15,  # max. number of function evaluations; for this example set to a low number
                     "cs": cs,  # configuration space
                     "deterministic": "true"
                     })

# Example call of the function
# It returns: Status, Cost, Runtime, Additional Infos
def_value = test_RS(cs.get_default_configuration())
print("Default Value: %.2f" % def_value)

# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC4HPO(scenario=scenario,
                rng=np.random.RandomState(42),
                tae_runner=test_RS)

incubent = smac.optimize()

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-04-19_02:38:38_928141
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 3 configurations
INFO:smac.facade.smac_hpo_facade.SMAC4HPO:<class 'smac.facade.smac_hpo_facade.SMAC4HPO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent


Default Value: 5.00
Optimizing! Depending on your machine, this might take a few minutes.


INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 5.0000
INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 0.054041 sec, available: 0.000010 sec)
INFO:smac.intensification.intensification.Intensifier:Challenger (4.0000) is better than incumbent (5.0000) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:
INFO:smac.intensification.intensification.Intensifier:  x0 : 0.895 -> 0.8475
INFO:smac.intensification.intensification.Intensifier:  x1 : 30 -> 45
INFO:smac.intensification.intensification.Intensifier:  x2 : 3 -> 2
INFO:smac.intensification.intensification.Intensifier:  x3 : 16 -> 23
INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 0.024328 sec, available

In [125]:
# print("Optimized Value: %.2f" % (inc_value))

# # We can also validate our results (though this makes a lot more sense with instances)
# smac.validate(config_mode='inc',  # We can choose which configurations to evaluate
#               # instance_mode='train+test',  # Defines what instances to validate
#               repetitions=100,  # Ignored, unless you set "deterministic" to "false" in line 95
#               n_jobs=1)  # How many cores to use in parallel for optimization

INFO:smac.utils.validate.Validator:Collected 0 runs from 1 configurations on 1 instances with 1 repetitions. Reusing 1 runs from given runhistory.
INFO:smac.utils.validate.Validator:Saving validation-results in smac3-output_2021-04-19_02:38:38_928141/run_1608637542/validated_runhistory.json


In [ ]:
# logging.basicConfig(level=20)  # 10: debug; 20: info
# m,soluce = heuristic_FFD(c,items)
# x, cost, _ = fmin_smac(func=test_RT,  # function
#               x0=[5],  # default configuration
#               bounds=[(1, 10)],  # limits
#               maxfun=5,  # maximum number of evaluations
#               rng=3)  # random seed
# print("Optimum at {} with cost of {}".format(x, cost))

# **Tests**

SCHOLL Instance


In [ ]:
datasets = ['HARD0.txt','HARD1.txt','HARD2.txt','HARD3.txt','HARD4.txt','HARD5.txt','HARD6.txt']

In [ ]:
def instance(inst):
  return [int(line.strip()) for line in open(inst, 'r')]

In [ ]:
print(heuristic_FFD(c,items)[0])

In [ ]:
print(heuristic_FFI(c,items)[0])

In [ ]:
m,soluce = heuristic_FFD(c,items)
RT(soluce,m,100)

In [ ]:
m,soluce = heuristic_FFD(c,items)
alpha = 0.94
t = 100
it_palier = 100
k_arret = 100
RS(soluce,m,alpha,t,it_palier,k_arret)

In [ ]:
dt = datasets[0]
dt = instance(dt)
c = dt[1]
items = dt[2:]
c

# **Visualisation**

# **Hyper-Heuristique**